<a href="https://colab.research.google.com/github/HarmanBhutani/ML_projects/blob/main/flowers_tf_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
assert tf.__version__.startswith('2')

import os
import numpy as np
import matplotlib.pyplot as plt

In [15]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.tgz", 
                                   extract=True)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

In [16]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [17]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 5))

In [18]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [19]:
!cat labels.txt

daisy
dandelion
roses
sunflowers
tulips

In [20]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

In [21]:
base_model.trainable = False

In [22]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 2,626,821
Trainable params: 368,837
Non-trainable params: 2,257,984
_________________________________________________________________


In [25]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 4


In [26]:
epochs = 10

history = model.fit(train_generator, 
                    steps_per_epoch=len(train_generator), 
                    epochs=epochs, 
                    validation_data=val_generator, 
                    validation_steps=len(val_generator))

Epoch 1/10
46/46 [==============================] - 132s 3s/step - loss: 0.8847 - accuracy: 0.7230 - val_loss: 0.5086 - val_accuracy: 0.8317
Epoch 2/10
46/46 [==============================] - 132s 3s/step - loss: 0.3184 - accuracy: 0.8966 - val_loss: 0.4739 - val_accuracy: 0.8468
Epoch 3/10
46/46 [==============================] - 132s 3s/step - loss: 0.1863 - accuracy: 0.9319 - val_loss: 0.4068 - val_accuracy: 0.8605
Epoch 4/10
46/46 [==============================] - 132s 3s/step - loss: 0.1144 - accuracy: 0.9690 - val_loss: 0.4363 - val_accuracy: 0.8577
Epoch 5/10
46/46 [==============================] - 131s 3s/step - loss: 0.0766 - accuracy: 0.9826 - val_loss: 0.5057 - val_accuracy: 0.8564
Epoch 6/10
46/46 [==============================] - 132s 3s/step - loss: 0.0445 - accuracy: 0.9925 - val_loss: 0.4494 - val_accuracy: 0.8741
Epoch 7/10
46/46 [==============================] - 132s 3s/step - loss: 0.0330 - accuracy: 0.9946 - val_loss: 0.4850 - val_accuracy: 0.8646
Epoch 8/10
46